# Занятие 1: HTTP-запросы, JSON, API.

## HTTP запросы

### Что такое HTTP запросы?
![http](https://httpwg.org/asset/http.svg)

Протокол HTTP предназначен для передачи данных. Именно он используется браузером, если нужно посмотреть мемы, видосики и все такое. Мы же можем его использовать чтобы получить или отправить какие-то данные с или на сервер.

Обмен сообщениями между сервером и клиентом проиходит по принципу "запрос-ответ". Клиент совершает некий запрос и получает на него ответ сервера.

Структура HTTP запроса:

* Строка запроса – указывает метод передачи, URL-адрес, к которому нужно обратиться и версию протокола HTTP.
* Заголовки – описывают тело сообщений, передают различные параметры и др. сведения и информацию.
* Тут пустая строка в качестве разделителя.
* Тело сообщения  — это сами данные, которые передаются в запросе.  Тело сообщения – это необязательный параметр и может отсутствовать

Например, если мы хотим получить статью википеди про http, то наш браузер сделает следующий запрос:
    
    GET /wiki/HTTP HTTP/1.1
    Host: ru.wikipedia.org
    User-Agent: Mozilla/5.0 (X11; U; Linux i686; ru; rv:1.9b5) Gecko/2008050509 Firefox/3.0b5
    Accept: text/html
    Connection: close
    (пустая строка)  
        
Первые 2 строки это строка запроса, все оставшиеся - заголовки. В этом запросе отсутствует тело.
Ответ на этот запрос будет следующий:

    HTTP/1.1 200 OK
    Date: Wed, 11 Feb 2009 11:20:59 GMT
    Server: Apache
    X-Powered-By: PHP/5.2.4-2ubuntu5wm1
    Last-Modified: Wed, 11 Feb 2009 11:20:59 GMT
    Content-Language: ru
    Content-Type: text/html; charset=utf-8
    Content-Length: 1234
    Connection: close
    (пустая строка)
    (запрошенная страница в HTML)

### На питоне

На мой взгляд, все, что сверху, выглядит, мягко говоря, сложновато.

Во-первых, вместо всего этого можно писать только URL.

Во-вторых, на питоне такие запросы будут выглядеть гораздо более удобно.

In [ ]:
# импортируем библиотеку, с помощью которой будем делать запросы
# если у вас нет библиотеки, установите её вот этой строчкой:
# !pip install requests
import requests 

In [ ]:
# GET-запрос просто получает html-код странички

responce = requests.get(url="http://sereja.me")
print(responce.text)

In [ ]:
print(responce.status_code)

Код 200 - это редкий случай, когда запрос произошел успешно. Этот код может быть равен [разным](https://http.cat/) значениям. Например, 404 - это когда запрос не успешен из-за отсутствия запрашиваемого ресурса.

### Еще примеры
Напишем функцию, которая принимает дату, и скачивает сырую страницу, на которой есть все футбольные матчи в этот день.

In [ ]:
def extract_footbal_matches_html(year, month, day):
    url = 'https://www.sports.ru/football/match/{}-{}-{}/'.format(year, month, day)
    response = requests.get(url)
    return response.text

extract_footbal_matches_html(2009, 4, 21)

Напишем функцию, которая получает страничку с информацией про место, к которому привязано IP.

In [ ]:
def extract_by_IP_html(IP):
    url = 'https://ipapi.co/{}/'.format(IP)
    response = requests.get(url)
    return response.text

extract_by_IP_html('91.134.227.240')

### Попробуйте сами
Выберите сайт и посмотрите как он выглядит в виде html-кода

### HTTP методы

![get vs post](https://i.ibb.co/mvPrFPs/getpost.jpg)

Чаще всего запрос - это метод GET или метод POST. Обычно GET - это запрос без изменения данных, а POST - это запрос с изменениями.

Мы пока делали только GET-запросы, именно они скачивают код страницы (на самом деле просто ваш браузер как код страницы воспринимает ровно то, что вернул GET-запрос).


### POST
Применяется для передачи пользовательских данных заданному ресурсу. Например, в блогах посетители обычно могут вводить свои комментарии к записям в HTML-форму, после чего они передаются серверу методом POST и он помещает их на страницу. При этом передаваемые данные (в примере с блогами — текст комментария) включаются в тело запроса. Аналогично с помощью метода POST обычно загружаются файлы на сервер.

Часто пост-запрос просто заполняет форму на сайте. Посмотрим на пример пост-запроса, который заполняют форму авторизации на сайте informatics, например.

In [ ]:
# сделать POST запрос с помощью библиотеки requests тоже просто

# вставьте сюда свои логин и пароль
DATA = {'username' : '...',
        'password' : '...'}

url = "https://informatics.mccme.ru/login/index.php"
r = requests.post(url, DATA)
html = r.text

print(html)

## JSON вместо HTML

![JSON Statham](https://pics.me.me/json-statha-meanwhile-43613192.png)

Заметьте, что если бы вы хотели извлечь все матчи из sports.ru за какой-то день, то вам пришлось бы сидеть и парсить огромный HTML-код, то есть научиться извлекать полезную и нужную вам информацию (список матчей, их времени, участников и итогог счета, например). Иногда приходится так делать, и этим мы скорее всего и займемся на следующем занятии.

Но многие сервисы знают, что ими пользуются не только пользователи через интерфейс, но и скрипты. И упрощают для них работу. А именно, создают специальный формат URL, на запрос по которому вместо html-кода выдается строка с данными в удобном формате, которую не надо сложно парсить.

Например, сайт ipapi.co называется так не случайно. Он умеет не только красиво показывать данные в интерфейсе по адресу [ipapi.co/91.134.227.240/](http://ipapi.co/91.134.227.240/), но и возвращать только полезную информацию в формате JSON по адресу [ipapi.co/91.134.227.240/json](https://ipapi.co/91.134.227.240/json)

In [ ]:
def extract_by_ip_json(ip):
    url = 'https://ipapi.co/{}/json'.format(ip)
    response = requests.get(url)
    return response.text

print(extract_by_ip_json('91.134.227.240'))

Согласитесь, что это гораздо более легко обрабатываемая строчка, чем огромный html-код :) Этот формат называется и называется JSON.

### Более подробно про JSON
[JSON](https://ru.wikipedia.org/wiki/JSON) - это очень удобный универсальный формат данных, состоящий из списков, словарей, строчек, чисел, true, false и null-ов. Причем списки и словари могут быть любой вложенности: это может быть словарь, где значениями являются другие словари, в некоторых из которых значениями являются списки словарей и так далее.

Перевод питоновских словарей/списков в json и обратно умеет делать библиотека json.

In [ ]:
# если у вас нет библиотеки, установите её вот этой строчкой:
# !pip install json
import json

### Функция json.loads
loads = load string, принимает на вход строку, возвращает данные

In [ ]:
s = extract_by_ip_json('91.134.227.240')
data = json.loads(s)
data

In [ ]:
# это обычный словарь
print(data['longitude'], data['latitude'])
print(data['country_name'])
print(data['city'])

### Функция json.dumps
dumps = dump string, принимает на вход данные, возращает строку

In [ ]:
s = json.dumps(data)
print(s)

### Работа с файлами
Те же функции, но без s (то есть load и dump) - вместо превращения из/в строку считывают/записывают из файла.

In [ ]:
# dump
with open('ip.json', 'w') as f:
    json.dump(data, f)

In [ ]:
# load
with open('ip.json', 'r') as f:
    data = json.load(f)
print(data['region'])

### Примеры
Давайте напишем функцию, которая по IP возвращает название города.

In [ ]:
# Первый способ
# Сделаем то же самое

def extract_by_ip_json(ip):
    url = 'https://ipapi.co/{}/json'.format(ip)
    response = requests.get(url)
    return json.loads(response.text)

def extract_city_by_ip_1(ip):
    data = extract_by_ip_json(ip)
    return data['city']

print(extract_city_by_ip_1('91.134.227.240'))

In [ ]:
# Второй способ
# Заметим в описании сайта (https://ipapi.co/#api),
# что можно делать запросы сразу про название города

def extract_city_by_ip_2(ip):
    url = 'https://ipapi.co/{}/city'.format(ip)
    response = requests.get(url)
    return response.text

print(extract_city_by_ip_2('91.134.227.240'))

## API

![API](https://static.consolia-comic.com/comics/what-is-an-api.png)

Такой http-интерфейс для общения с сервисом называется API. Сайтов, которые их предоставляют, очень много. Как и на [ipapi.co/#api](https://ipapi.co/#api), на них всегда есть страничка с документацией, объясняющая, на какие запросы умеет отвечать сервис, и в каком формате делать такие запросы.

[Список сервисов с API 1](https://www.programmableweb.com/apis/directory)

[Список сервисов с API 2](https://github.com/public-apis/public-apis/blob/master/README.md) (тут отмечены те, в которых не нужна авторизация).

[Список сервисов с API 3](https://shkspr.mobi/blog/2016/05/easy-apis-without-authentication/) (только те, в которых не была нужна авторизация на момент публикации)

![front vs back](https://i.imgur.com/cNECvO3.png)

### Задание
* Найдите всех героев Dota 2, у которых больше двух ног.

* Выведите для каждого его имя

* А также имя того героя, против которого у этого героя самый высокий процент побед в последнее время, если учитывать только героев, против которого он сыграл хотя бы 20 матчей.

### API с авторизацией

![api key](https://pics.me.me/api-locksmiths-131-key-24-7-service-alarms-alarm-monitoring-37101506.png)

На самом деле API, которые мы показывали (ipapi и opendota) - очень редкий случай, когда сервис не запрашивает авторизацию. В opendata, кстати, есть авторизация, и она увеличивает лимит на число запросов в месяц по сравнению с обычными запросами.

Но практически у всех больших сервисов (типа Google, Telegram, Twitter) взаимодействие с API происходит с помощью авторизации.

Авторизация преследует много разных целей, и у нее есть много разных реализаций. Но в целом она выглядит так: тебе как пользователю сервиса (например, гугла), выдается личный **токен** по твоему запросу (обычно даже несколько токенов). Это просто длинная строка, которую ты прикладываешь к запросу, чтобы сервис знал, кто его послал.

Так можно отслеживать тех, кто злоупотребляет запросами и посылает их слишком много, а главное - с токеном ты можешь получить доступ к данным своего пользователя. Например, можно скачать всю свою переписку в VK, или даже посылать сообщения в VK через скрипты.

Давайте попробуем подключиться по API, например, к Яндекс.Переводчику.

* Получите токен [тут](https://translate.yandex.ru/developers/keys)
* Сохраните его в переменную
* Запустите код

In [ ]:
YANDEX_TRANSLATE_TOKEN = ''

def get_all_langs():
    url = 'https://translate.yandex.net/api/v1.5/tr.json/getLangs?key={}&ui=ru'.format(YANDEX_TRANSLATE_TOKEN)
    return json.loads(requests.get(url).text)

langs = get_all_langs()
langs['langs']

In [ ]:
def translate_sentence(sentence):
    url = 'https://translate.yandex.net/api/v1.5/tr.json/translate?key={}&lang=en-ru&text={}'.format(YANDEX_TRANSLATE_TOKEN,
                                                                                                     sentence)
    return json.loads(requests.get(url).text)['text']

translate_sentence('Somebody once told me the world is gonna roll me.')

Заметьте, какой длинный получается URL! И всё из-за параметров запроса вида key1=value1&key2=value2&... На самом деле в функцию requests.get можно передать URL без параметров, а сами параметры перечислить в отдельном аргументе-словаре, это намного красивее (и мы так делали, когда посылали POST-запрос).

In [ ]:
def translate_sentence(sentence):
    url = 'https://translate.yandex.net/api/v1.5/tr.json/translate'
    params = {'key': YANDEX_TRANSLATE_TOKEN,
              'lang': 'en-ru',
              'text': sentence} 
    return json.loads(requests.get(url, params).text)['text']

translate_sentence('Somebody once told me the world is gonna roll me.')

### Задание
Напишите функцию, которая по названию города находит текущую температуру в нем.

* Зарегистрируйтесь на сайте https://home.openweathermap.org/users/sign_up
* Найдите свой токен вот здесь https://home.openweathermap.org/api_keys
* Вставьте токен в переменную
* Почитайте документацию, чтобы понять, как пользоваться API https://openweathermap.org/current
* Напишите функцию, которая выводит текущую погоду в заданном городе
* Возможно, надо подождать немного, чтобы ключ начал работать

In [ ]:
OPENWEATHER_TOKEN = '...'

## API  через библиотеку

![library](http://cecilialiao.com/wp-content/uploads/2015/06/mlwdc.jpg)

И наконец последняя стадия развития человечества: авторизация на такие сайты настолько сложная, а запросы настолько длинные и непонятные, что люди пишут целые небольшие библиотеки, в которых уже реализованы все функции, делающие http-запросы. Практически у всех крупных компаний (Гугл, Фейсбук, Телеграм) есть такие библиотеки.

Например, при помощи библиотеки [vk_api](https://github.com/python273/vk_api) можно делать разные действия ВКонтакте:

In [ ]:
# если библиотека не импортируется, может помочь рестарт кернела, мне помогло 
# !pip install vk_api
import vk_api 

In [ ]:
VK_LOGIN = '...' # ваша почта или телефон
VK_PASSWORD = '...'

# авторизация 
vk_session = vk_api.VkApi(VK_LOGIN, VK_PASSWORD)
vk_session.auth()

# получение экзмепляра класса, реализующего API 
vk = vk_session.get_api() 

# вывести ваше имя и фамилию
info = vk.account.getProfileInfo()
print(info['first_name'], info['last_name'])

In [ ]:
# разместить на Вашей стене сообщение Hello world! 
print(vk.wall.post(message='Hello world!'))

In [ ]:
def get_user_id_by_name(name): 
    search_results = vk.users.search(q=name)
    return search_results['items'][0]['id']

get_user_id_by_name('Павел Дуров')

In [ ]:
get_user_id_by_name('Андрей Гаркавый')

In [ ]:
get_user_id_by_name('Дмитрий Медведев')

### Еще пример: Google

Теперь научися подключаться к гугл апи, это немного тяжелее.

Например, давайте научимся записывать что-нибудь в гугл-таблицы.

![google](http://www.ohiofi.com/blog/wp-content/uploads/2014/05/resized_the-most-interesting-man-in-the-world-meme-generator-i-don-t-always-excel-but-when-i-do-it-s-at-spreadsheets-7d56e8-239x300.jpg)


In [ ]:
# !pip install google-api-python-client
# !pip install oauth2client
from googleapiclient.discovery import build
from oauth2client import file, client, tools

In [ ]:
# На что просить права
SCOPES = [
    'https://www.googleapis.com/auth/spreadsheets',
    'https://www.googleapis.com/auth/drive'
]

# Функция, настраивающая соединение с API
def connect_to_google_api():
    store = file.Storage('token.json')
    flow = client.flow_from_clientsecrets('credentials.json', SCOPES)
    flags = tools.argparser.parse_args(args=[])
    return tools.run_flow(flow, store, flags)

# создаем файл, потому что он будет использоваться
!touch token.json

[Здесь](https://console.developers.google.com/apis/credentials) можно получить credentials.json для гугла:
* "Cоздать учетные данные"
* "Идентификатор клиента OAuth"
* Введите название приложения
* Выберите тип приложения
* Скачайте json файл и назовите его credentials.json

In [ ]:
creds = connect_to_google_api()

In [ ]:
from httplib2 import Http

# создаем объект, который умеет вызывать методы API
service = build('sheets', 'v4', http=creds.authorize(Http()))

Создайте на Google.Drive пустую таблицу и запишите сюда ее id.

Давайте начимся писать в ней данные прямо из этого ноутбука.

In [ ]:
TABLE_ID = '...' # id - это огромная строка в url таблицы

def update_table(table_id, values, range_name="Лист1", valueInputOption='RAW'):
    return service.spreadsheets().values().update(
        spreadsheetId=table_id,
        range=range_name,
        valueInputOption=valueInputOption,
        body={'values': values}, 
    ).execute()

update_table(
    table_id=TABLE_ID,
    values=[['Avatar', 2787965087, 2009],
            ['Avengers: Endgame', 2751645367, 2019],
            ['Titanic', 2187463944, 1997]]
)

### Задание
Возможно на дом, для желающих.

* Создайте пустую гугл-таблицу.
* Напишите функцию, которая выкачивает ваши последние 50 постов в ленте вк.
* Напишите функцию, которая извлекает из них структурированную табличную информацию (текст, автор, количество просмотров, лайков, репостов, комментариев).
* Напишите функцию, которая делает все это и заливает в вашу гугл-таблицу.

### Замечание про то, как хранить токены и пароли

![пароль](https://pbs.twimg.com/media/DLiNe_bXUAALfCU.jpg:large)

Никогда, никогда не храните пароль/токен прямо в коде типа

In [35]:
PASSWORD = 'best_password1234'

# (в этом занятии я вот например стёр везде свои пароли :D)

Особенно если вы заливаете свой код куда-либо (на гитхаб).

Все токены, пароли, ключи авторизации удобно хранить в отдельном от кода файле (кстати, формат JSON бы подошел), который никогда-никогда никуда не заливать (тем более в открытый доступ), чтобы ключи хранились только локально у вас на компьютере, либо на сервере. Помните, что если вы уже залили что-то на гитхаб, оно уже навсегда останется там в истории, еси вы только не удалите свой репозиторий.

При этом ваш основной скрипт, запускающий программу, должен просто читать этот файлик и использовать хранящиеся там пароли.

### gitignore

![gitignore](https://pics.esmemes.com/gitignore-2-you-were-the-chosen-one-ironic-32082411.png)

Чтобы файл с ключами не попал в проект, когда вы заливаете локальный проект на гит (делаете commit и push через консоль или приложение), вам нужно добавить в проект файл с названием ".gitignore". 

Если вы напишете в файл ".gitignore" строчку с названием файла, то этот файл будет игнорироваться гитом, и не попадет ни в один коммит. 

Рекомендую проверить сначала на другом файле, а не сразу бежать заливать настоящий пароль :) Если вы заливаете пофайлово через интерфейс на сайте, то такой проблемы нет.

![мем1](https://media.giphy.com/media/fxesWuvpUdKClpRoD4/giphy.gif)
